In [55]:
import pandas as pd
import numpy as np 
from tensorflow import keras 
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense,\
    Flatten, Conv2D, MaxPool2D
import matplotlib.pyplot as plt
from keras.models import load_model   
import cv2
from PIL import Image    

In [3]:
def load_data(validation_split):
    
    # Load data from csv file into data frame, drop all rows that have missing values
    data_frame = pd.read_csv('../data/training.csv')
    print(f'Number of samples: {data_frame["Image"].count()}')
    data_frame = data_frame.dropna()
    print(f'Number of samples without any missing value: {data_frame["Image"].count()}')

    # Convert the rows of the image column from pixel values separated by spaces to numpy arrays
    data_frame["Image"] = data_frame["Image"].apply(lambda img: np.fromstring(img, sep=" "))

    # Create numpy matrix from image column by stacking the rows vertically
    X_data = np.vstack(data_frame["Image"].values)

    # Normalize pixel values to (0, 1) range
    X_data = X_data / 255

    # Convert to float32, which is the default for Keras
    X_data = X_data.astype("float32")

    # Reshape each row from one dimensional arrays to (height, width, num_channels) = (96, 96, 1)
    X_data = X_data.reshape(-1, 96, 96, 1)

    # Extract labels representing the coordinates of facial landmarks
    Y_data = data_frame[data_frame.columns[:-1]].values

    # Normalize coordinates to (0, 1) range
    Y_data = Y_data / 96
    Y_data = Y_data.astype("float32")

    # Shuffle data
    X_data, Y_data = shuffle(X_data, Y_data)

    # Split data into training set and validation set
    split_index = int(X_data.shape[0] * (1 - validation_split))
    X_train = X_data[:split_index]
    Y_train = Y_data[:split_index]
    X_val = X_data[split_index:]
    Y_val = Y_data[split_index:]

    return X_train, Y_train, X_val, Y_val



In [57]:
data = pd.read_csv('../data/training.csv')

In [110]:
#get columns names to search for needed columns index 
target_variables_feature_name = list(data.columns)
target_variables_feature_name

['left_eye_center_x',
 'left_eye_center_y',
 'right_eye_center_x',
 'right_eye_center_y',
 'left_eye_inner_corner_x',
 'left_eye_inner_corner_y',
 'left_eye_outer_corner_x',
 'left_eye_outer_corner_y',
 'right_eye_inner_corner_x',
 'right_eye_inner_corner_y',
 'right_eye_outer_corner_x',
 'right_eye_outer_corner_y',
 'left_eyebrow_inner_end_x',
 'left_eyebrow_inner_end_y',
 'left_eyebrow_outer_end_x',
 'left_eyebrow_outer_end_y',
 'right_eyebrow_inner_end_x',
 'right_eyebrow_inner_end_y',
 'right_eyebrow_outer_end_x',
 'right_eyebrow_outer_end_y',
 'nose_tip_x',
 'nose_tip_y',
 'mouth_left_corner_x',
 'mouth_left_corner_y',
 'mouth_right_corner_x',
 'mouth_right_corner_y',
 'mouth_center_top_lip_x',
 'mouth_center_top_lip_y',
 'mouth_center_bottom_lip_x',
 'mouth_center_bottom_lip_y',
 'Image']

In [109]:
target_variables_feature_name[15]

'left_eyebrow_outer_end_y'

In [113]:
print(
    f'left_eye_outer_corner_x index: {target_variables_feature_name.index("left_eye_outer_corner_x")}\n',
    f'left_eye_outer_corner_y index: {target_variables_feature_name.index("left_eye_outer_corner_y")}\n', #4
    f'right_eye_outer_corner_x index: {target_variables_feature_name.index("right_eye_outer_corner_x")}\n',
    f'right_eye_outer_corner_y index: {target_variables_feature_name.index("right_eye_outer_corner_y")}\n',
    f'left_eyebrow_outer_end_y index: {target_variables_feature_name.index("right_eyebrow_outer_end_y")}\n',
    )

left_eye_outer_corner_x index: 6
 left_eye_outer_corner_y index: 7
 right_eye_outer_corner_x index: 10
 right_eye_outer_corner_y index: 11
 left_eyebrow_outer_end_y index: 19



In [4]:
#split the data into train and test 
X_train, y_train, X_val, y_val = load_data(validation_split = 0.15)

Number of samples: 7049
Number of samples without any missing value: 2140


In [5]:
#landmarks
y_train

array([[0.708875  , 0.37780148, 0.30892646, ..., 0.6785956 , 0.48741913,
        0.8240294 ],
       [0.7011118 , 0.39410558, 0.32081988, ..., 0.7022236 , 0.49569565,
        0.82713664],
       [0.6832875 , 0.381525  , 0.32855   , ..., 0.708325  , 0.53524375,
        0.86195   ],
       ...,
       [0.67078733, 0.4090699 , 0.3596746 , ..., 0.839437  , 0.47497538,
        0.8574032 ],
       [0.6754487 , 0.3689263 , 0.30589104, ..., 0.69551283, 0.5178846 ,
        0.8244295 ],
       [0.65833664, 0.38317   , 0.31077334, ..., 0.77008   , 0.49767   ,
        0.83729666]], dtype=float32)

In [6]:
#images
X_train

array([[[[0.4627451 ],
         [0.49019608],
         [0.4117647 ],
         ...,
         [0.9490196 ],
         [0.68235296],
         [0.50980395]],

        [[0.4862745 ],
         [0.4745098 ],
         [0.4       ],
         ...,
         [0.972549  ],
         [0.72156864],
         [0.52156866]],

        [[0.49019608],
         [0.44313726],
         [0.39607844],
         ...,
         [1.        ],
         [0.8509804 ],
         [0.57254905]],

        ...,

        [[0.45490196],
         [0.34901962],
         [0.29803923],
         ...,
         [0.4117647 ],
         [0.56078434],
         [0.75686276]],

        [[0.34509805],
         [0.3764706 ],
         [0.3372549 ],
         ...,
         [0.49803922],
         [0.6039216 ],
         [0.7411765 ]],

        [[0.28235295],
         [0.33333334],
         [0.32941177],
         ...,
         [0.5254902 ],
         [0.65882355],
         [0.7529412 ]]],


       [[[0.25490198],
         [0.28235295],
         [0.31

In [8]:
def create_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(96, 96, 1), activation="relu", padding="same"))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (5, 5), activation="relu", padding="same"))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5), activation="relu", padding="same"))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(512, activation="relu"))
    model.add(Dense(30))
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model

In [9]:
def run_models(X_train, Y_train, X_val, Y_val, model, model_name, num_epochs):
    results_file = open("../output/results.txt", "w")
    histories = []
    
    # Train model
    history = model.fit(
        X_train, 
        Y_train, 
        batch_size = 200, 
        epochs = num_epochs, 
        validation_data = (X_val, Y_val)
    )           
        
    histories.append(history)

    # Evaluate
    final_train_loss = model.evaluate(X_train, Y_train, verbose=0)
    final_val_loss = model.evaluate(X_val, Y_val, verbose=0)
    results_file.write(model_name + " => final_train_loss: " + str(final_train_loss) + ", final_val_loss: " + str(final_val_loss) + "\n")

    # Save model
    model.save("../models/facial_landmark_model")       
        
    results_file.close()

In [10]:
NUM_EPOCHS = 150

# Run the model
cnn_model = create_cnn_model()   
run_models(X_train, y_train, X_val, y_val, cnn_model, "cnn", NUM_EPOCHS)

Epoch 1/150
10/10 [==============================] - 22s 494ms/step - loss: 0.4715 - val_loss: 0.0837
Epoch 2/150
10/10 [==============================] - 3s 306ms/step - loss: 0.0395 - val_loss: 0.0170
Epoch 3/150
10/10 [==============================] - 3s 305ms/step - loss: 0.0162 - val_loss: 0.0124
Epoch 4/150
10/10 [==============================] - 3s 305ms/step - loss: 0.0112 - val_loss: 0.0067
Epoch 5/150
10/10 [==============================] - 3s 305ms/step - loss: 0.0048 - val_loss: 0.0025
Epoch 6/150
10/10 [==============================] - 3s 305ms/step - loss: 0.0023 - val_loss: 0.0023
Epoch 7/150
10/10 [==============================] - 3s 306ms/step - loss: 0.0017 - val_loss: 0.0023
Epoch 8/150
10/10 [==============================] - 3s 305ms/step - loss: 0.0016 - val_loss: 0.0015
Epoch 9/150
10/10 [==============================] - 3s 307ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 10/150
10/10 [==============================] - 3s 305ms/step - loss: 0.0012 - val_l